In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,42.80,74,94,36.82,overcast clouds
1,1,Marinette,US,45.1000,-87.6307,68.05,94,100,10.36,mist
2,2,Bansang,GM,13.4333,-14.6500,92.50,46,87,6.93,overcast clouds
3,3,Bengkulu,ID,-3.8004,102.2655,75.88,89,78,4.05,light rain
4,4,Vaini,TO,-21.2000,-175.2000,68.16,100,20,3.44,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp =float(input("What is the minimum temperature you would like for your trip? " ))
max_temp =float(input("What is the maximum temperature you would like for your trip? " ))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bansang,GM,13.4333,-14.6500,92.50,46,87,6.93,overcast clouds
3,3,Bengkulu,ID,-3.8004,102.2655,75.88,89,78,4.05,light rain
11,11,Kalmunai,LK,7.4167,81.8167,75.33,86,93,8.19,overcast clouds
17,17,Butaritari,KI,3.0707,172.7902,82.02,72,95,13.06,overcast clouds
20,20,Hithadhoo,MV,-0.6000,73.0833,83.37,67,38,2.10,scattered clouds
25,25,Bandarbeyla,SO,9.4942,50.8122,79.39,76,27,32.17,scattered clouds
27,27,Jamestown,US,42.0970,-79.2353,79.02,62,80,1.99,broken clouds
29,29,Poum,NC,-20.2333,164.0167,75.40,83,8,8.34,clear sky
33,33,Coari,BR,-4.0850,-63.1414,80.92,75,95,1.07,overcast clouds
36,36,Pierre,US,44.3683,-100.3510,76.86,52,0,16.11,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
# preferred_cities_df.count()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
dropped_df = preferred_cities_df.dropna()
dropped_df.count()

City_ID                255
City                   255
Country                255
Lat                    255
Lng                    255
Max Temp               255
Humidity               255
Cloudiness             255
Wind Speed             255
Current Description    255
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = dropped_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bansang,GM,92.50,overcast clouds,13.4333,-14.6500,
3,Bengkulu,ID,75.88,light rain,-3.8004,102.2655,
11,Kalmunai,LK,75.33,overcast clouds,7.4167,81.8167,
17,Butaritari,KI,82.02,overcast clouds,3.0707,172.7902,
20,Hithadhoo,MV,83.37,scattered clouds,-0.6000,73.0833,
25,Bandarbeyla,SO,79.39,scattered clouds,9.4942,50.8122,
27,Jamestown,US,79.02,broken clouds,42.0970,-79.2353,
29,Poum,NC,75.40,clear sky,-20.2333,164.0167,
33,Coari,BR,80.92,overcast clouds,-4.0850,-63.1414,
36,Pierre,US,76.86,clear sky,44.3683,-100.3510,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row ["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel added to DF")
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel not found... skipping.
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel not found... skipping.
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel not found... skipping.
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added to DF
Hotel added t

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   255
Country                255
Max Temp               255
Current Description    255
Lat                    255
Lng                    255
Hotel Name             255
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))